Resources: 
https://www.youtube.com/watch?v=dZh_ps8gKgs&t=1251s
https://rockyshikoku.medium.com/how-to-use-tensorflow-object-detection-api-with-the-colab-sample-notebooks-477707fadf1b

# have to run through cmd in D drive!!!!!!!!!!!


In [1]:
!pip install pyparsing==2.4.7 
!pip install --upgrade witwidget -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-core 1.7.1 requires google-api-core<2.0.0dev,>=1.21.0, but you have google-api-core 2.11.0 which is incompatible.
google-cloud-core 1.7.1 requires google-auth<2.0dev,>=1.24.0, but you have google-auth 2.17.3 which is incompatible.


# Define Paths

In [2]:
#Define Paths:
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
WORKSPACE_PATH = 'Tensorflow/workspace'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/sign_model/pipeline.config'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
CHECKPOINT_PATH = MODEL_PATH+'/sign_model/'

# Create labels for our detections

- Define our label maps

In [3]:
#Define our label maps
labels = [{'name':'Stop', 'id':1}, #have to be > then 0
          {'name':'Yeild', 'id':2},
          {'name':'Danger', 'id':3},
          {'name':'Deadend', 'id':4},
          {'name':'Parking', 'id':5}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Create Tensorflow records for us to train with


<img src="pic/traintestrecord.jpg">

- Create train records

In [4]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}


Traceback (most recent call last):
  File "C:\Users\sophiechen\2022MDS\Block 6\Data-586\RoadSignDetection\Tensorflow\scripts\generate_tfrecord.py", line 61, in <module>
    label_map_dict = label_map_util.get_label_map_dict(args.labels_path)
  File "C:\Users\sophiechen\anaconda3\lib\site-packages\object_detection\utils\label_map_util.py", line 164, in get_label_map_dict
    label_map = load_labelmap(label_map_path)
  File "C:\Users\sophiechen\anaconda3\lib\site-packages\object_detection\utils\label_map_util.py", line 132, in load_labelmap
    with tf.gfile.GFile(path, 'r') as fid:
AttributeError: module 'tensorflow' has no attribute 'gfile'


- Create test records

In [5]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}


Traceback (most recent call last):
  File "C:\Users\sophiechen\2022MDS\Block 6\Data-586\RoadSignDetection\Tensorflow\scripts\generate_tfrecord.py", line 61, in <module>
    label_map_dict = label_map_util.get_label_map_dict(args.labels_path)
  File "C:\Users\sophiechen\anaconda3\lib\site-packages\object_detection\utils\label_map_util.py", line 164, in get_label_map_dict
    label_map = load_labelmap(label_map_path)
  File "C:\Users\sophiechen\anaconda3\lib\site-packages\object_detection\utils\label_map_util.py", line 132, in load_labelmap
    with tf.gfile.GFile(path, 'r') as fid:
AttributeError: module 'tensorflow' has no attribute 'gfile'


# Update pipline config 

<img src="pic/pipeline.jpg">

In [6]:
import tensorflow as tf

In [7]:
print(tf.__version__) #might have compability issues with cuda 

2.11.0


In [8]:
# copy model config to training Folder
import shutil
CUSTOM_MODEL_NAME = 'sign_model' 
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
shutil.copy(PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', MODEL_PATH+'/'+CUSTOM_MODEL_NAME)


A subdirectory or file Tensorflow\workspace\models\sign_model already exists.


'Tensorflow/workspace/models/sign_model\\pipeline.config'

In [9]:
!pip install object_detection

In [11]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [12]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
print(CONFIG_PATH)
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

Tensorflow/workspace/models/sign_model/pipeline.config


ParseError: 172:3 : Message type "object_detection.protos.TrainConfig" has no field named "fine_tune_checkpoint_version".

- Our pipeline config:


In [15]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

ParseError: 172:3 : Message type "object_detection.protos.TrainConfig" has no field named "fine_tune_checkpoint_version".

In [13]:
len(labels) #check how many label to detect

5

- Update our pipeline config

In [14]:
pipeline_config.model.ssd.num_classes = len(labels)  #number of class we have
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'  #use transfer learning to train faster from checkpoints
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

NameError: name 'pipeline_config' is not defined

In [ ]:
pipeline_config

# Training (NEW)

if re-train:
#delete check point in Tensotflow/model/sign_model if needed
#delete check point in Tensotflow/model/annotations if needed

In [ ]:
from pathlib import Path
import imghdr

<img src="pic/pretrainmodel.jpg">

<img src="pic/annotations.jpg">

- Model Training: Run in CMD!!! Copy and paste the following to CMD to start training the model.



In [ ]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

<img src="pic/train.jpg">

- Model summary: Run in CMD!!! Copy and paste the following to CMD to see the model performance



In [ ]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --checkpoint_dir={}/{}""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

<img src="pic/performance.jpg">

# Load pre-trained model

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/sign_model/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/sign_model/'

In [ ]:
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

#look at the model path and use the latest checkpoint!!!!
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-11')).expect_partial() 

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Test on Unseen Data

In [ ]:
from io import BytesIO
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
from PIL import Image, ImageDraw, ImageFont  

WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/sign_model/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/sign_model/'

configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

#look at the model path and use the latest checkpoint!!!!
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-11')).expect_partial() 

def load_image_into_numpy_array(path):
   img_data = tf.io.gfile.GFile(path, 'rb').read()
   image = Image.open(BytesIO(img_data))
   (im_width, im_height) = image.size
   return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def get_keypoint_tuples(eval_config):
   tuple_list = []
   kp_list = eval_config.keypoint_edge
   for edge in kp_list:
     tuple_list.append((edge.start, edge.end))
   return tuple_list

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image) #resize image to 300x300
    prediction_dict = detection_model.predict(image, shapes) #use detection model to predict
    detections = detection_model.postprocess(prediction_dict, shapes) #preprocess again
    return detections


category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')


image_dir = 'Tensorflow/workspace/images/test/' 
image_path = os.path.join(image_dir, '5.park18.jpg')
image_np = load_image_into_numpy_array(image_path) 
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections= detect_fn(input_tensor)
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}

detections['num_detections'] = num_detections
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.5,
            agnostic_mode=False)

plt.figure(figsize=(12,16)) 
plt.imshow(image_np_with_detections) 
plt.show()
# cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))

In [ ]:
category_index

# OpenCV (Main)

In [ ]:
print(cv2.__version__)

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import cv2 
import numpy as np
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/sign_model/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/sign_model/'

# Load pipeline 
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Load checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-11')).expect_partial() #look at the model path

# funtion that produce detection if we pass our image to
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image) #resize image to 300x300
    prediction_dict = detection_model.predict(image, shapes) #use detection model to predict
    detections = detection_model.postprocess(prediction_dict, shapes) #preprocess again
    return detections

category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')


#openCV
cap = cv2.VideoCapture(0) #0 for default camera
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break
        
print(cv2.__version__)
detections = detect_fn(input_tensor)
from matplotlib import pyplot as plt